In [11]:
from config import *
from db import *
from utilities import *

In [12]:
try:
    current_path = os.path.dirname(os.path.abspath(__file__))
except:
    current_path = '.'

In [13]:
driver = init_driver(gecko_driver, user_agent = user_agent, is_headless = False)

In [14]:
# https://www.walmart.ca/en/top-deals-in-electronics/N-33983
categories = ['top-deals-in-electronics/N-33983']

In [15]:
for category in categories:
    category_url = f"{walmart_base_url}/{category}"
    
    for page in range(1,4):
        page_url = f"{category_url}/page-{str(page)}"
        get_url(page_url, driver)
        
        products = get_products(driver)
        break